# **IoT-Based Disaster Prediction and Response System**

# Heatwave prediction Model

In [1]:
!pip install xgboost

In [2]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib

In [38]:
# Load and filter dataset
df = pd.read_csv("surface temperature anomalies by year.csv")
df = df[df['Unnamed: 0'] == 'United States']

# Extract only year columns
year_columns = [col for col in df.columns if col.isnumeric()]
df_us = df[year_columns]

# Transpose to get one row per year
df_transposed = df_us.transpose().reset_index()
df_transposed.columns = ['Year', 'Temp_Anomaly']

# Clean and convert
df_transposed['Year'] = pd.to_numeric(df_transposed['Year'], errors='coerce')
df_transposed['Temp_Anomaly'] = pd.to_numeric(df_transposed['Temp_Anomaly'], errors='coerce')
df_transposed.dropna(inplace=True)

# Label heatwave years (top 10% temp anomalies)
threshold = df_transposed['Temp_Anomaly'].quantile(0.90)
df_transposed['Heatwave'] = (df_transposed['Temp_Anomaly'] >= threshold).astype(int)

# Prepare data
X = df_transposed[['Year', 'Temp_Anomaly']]
y = df_transposed['Heatwave']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "heatwave_xgb_model_us_only.pkl")

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:03:04] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


['heatwave_xgb_model_us_only.pkl']

In [39]:
# Evaluate
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         1

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

Confusion Matrix:
[[10  0]
 [ 0  1]]


# Wildfire Prediction Model

In [40]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# Load dataset
df = pd.read_csv("CA_Weather_Fire_Dataset_1984-2025.csv")

# Keep only 3 relevant features and target
df = df[['MAX_TEMP', 'AVG_WIND_SPEED', 'WIND_TEMP_RATIO', 'FIRE_START_DAY']].dropna()

# Rename to match IoT input convention
df = df.rename(columns={
    'MAX_TEMP': 'temp',
    'AVG_WIND_SPEED': 'wind_speed',
    'WIND_TEMP_RATIO': 'air_quality'
})

# Define features and target
X = df[['temp', 'wind_speed', 'air_quality']]
y = df['FIRE_START_DAY'].astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost model
model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

# Save model
joblib.dump(model, "wildfire_xgb_model.pkl")


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [02:05:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


['wildfire_xgb_model.pkl']

In [41]:
# Evaluate model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.75      0.82      0.78      1989
           1       0.56      0.46      0.51      1007

    accuracy                           0.70      2996
   macro avg       0.66      0.64      0.65      2996
weighted avg       0.69      0.70      0.69      2996

Confusion Matrix:
 [[1627  362]
 [ 539  468]]


# Final Prediction with iot simulated dataset

In [45]:
import pandas as pd
import joblib

# Load your IoT data
df = pd.read_csv("feed (1).csv")

# Rename columns to match model input
df = df[['field1', 'field3', 'field4']].dropna()
df.columns = ['temp', 'wind_speed', 'air_quality']

# Add year for heatwave prediction
df['Year'] = 2025  # Simulated year of observation
df['Temp_Anomaly'] = df['temp']  # Match heatwave model's expected feature

# Load trained ML models
wildfire_model = joblib.load("wildfire_xgb_model.pkl")
heatwave_model = joblib.load("heatwave_xgb_model_us_only.pkl")

# Run wildfire prediction
df['Wildfire_Predicted'] = wildfire_model.predict(
    df[['temp', 'wind_speed', 'air_quality']]
)

# Run heatwave prediction
df['Heatwave_Predicted'] = heatwave_model.predict(
    df[['Year', 'Temp_Anomaly']]
)

In [46]:
# Display predictions
print(df[['temp', 'wind_speed', 'air_quality', 'Wildfire_Predicted', 'Heatwave_Predicted']].head())

   temp  wind_speed  air_quality  Wildfire_Predicted  Heatwave_Predicted
0  42.6          23         3104                   0                   1
1  42.6          99         3628                   0                   1
2  42.6          64         3628                   0                   1
3  12.9          64         2603                   0                   1
4  52.2           2         2603                   0                   1


In [48]:
# Save predictions to CSV
df.to_csv("iot_wildfire_heatwave_predictions.csv", index=False)
print("\nPredictions saved to 'iot_wildfire_heatwave_predictions.csv'")


Predictions saved to 'iot_wildfire_heatwave_predictions.csv'


# E-Mail Alert

In [53]:
# Alert Summary
wildfire_count = df['Wildfire_Predicted'].sum()
heatwave_count = df['Heatwave_Predicted'].sum()

alert_lines = []

if wildfire_count > 0:
    alert_lines.append(f"🔥 potential wildfire detected.")
if heatwave_count > 0:
    alert_lines.append(f"🌡️ potential heatwave detected.")

# Send Email Alert
def send_email_alert(subject, body):
    msg = EmailMessage()
    msg.set_content(body)
    msg['Subject'] = subject
    msg['From'] = "iotmailcis600@gmail.com"
    msg['To'] = "rohiniskumar17@gmail.com"

    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.login("iotmailcis600@gmail.com", "imgn ktuz udai gpvn")
    server.send_message(msg)
    server.quit()
    print("Email alert sent!")

# Trigger Alert if Needed
if alert_lines:
    body = "\n".join(alert_lines)
    send_email_alert("🚨 IoT Disaster Alert!", body)
else:
    print("No alerts detected — no email sent.")

# Save predictions to CSV
df.to_csv("iot_predictions_with_alerts.csv", index=False)
print("Predictions saved to 'iot_predictions_with_alerts.csv'")


Email alert sent!
Predictions saved to 'iot_predictions_with_alerts.csv'
